In [ ]:
#@title **Setup** (einmal ausführen) { display-mode: "form" }
#@markdown Installiert benötigte Pakete und lädt den Code.

import sys
import os

# Pakete installieren
!pip install -q pyyaml requests plotly networkx kaleido

# Arbeitsverzeichnis erstellen
!mkdir -p /content/triple-colab/src
!mkdir -p /content/triple-colab/output_json
!mkdir -p /content/triple-colab/logs

# Python-Path erweitern
if '/content/triple-colab/src' not in sys.path:
    sys.path.insert(0, '/content/triple-colab/src')

# file_client.py erstellen
file_client_code = '''"""Datei-Client für XML/TXT-Dateien mit TEI-Optimierung."""
import os
import xml.etree.ElementTree as ET
from typing import Optional
import logging

logger = logging.getLogger(__name__)

class FileClient:
    """Client zum Lesen von XML/TXT-Dateien mit TEI-Optimierung."""
    
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.file_extension = os.path.splitext(file_path)[1].lower()
    
    def read_content(self) -> Optional[str]:
        """Liest den Dateiinhalt und extrahiert relevanten Text."""
        try:
            if self.file_extension == ".xml":
                return self._read_xml()
            elif self.file_extension == ".txt":
                return self._read_txt()
            else:
                logger.warning(f"Nicht unterstütztes Dateiformat: {self.file_extension}")
                return None
        except Exception as e:
            logger.error(f"Fehler beim Lesen von {self.file_path}: {e}")
            return None
    
    def _read_xml(self) -> Optional[str]:
        """Liest XML-Datei und extrahiert TEI-Body-Text."""
        try:
            tree = ET.parse(self.file_path)
            root = tree.getroot()
            namespaces = {"tei": "http://www.tei-c.org/ns/1.0"}
            body = root.find(".//tei:body", namespaces)
            if body is not None:
                return self._extract_text_from_element(body)
            body = root.find(".//body")
            if body is not None:
                return self._extract_text_from_element(body)
            return self._extract_text_from_element(root)
        except ET.ParseError as e:
            logger.error(f"XML-Parse-Fehler in {self.file_path}: {e}")
            return None
    
    def _read_txt(self) -> Optional[str]:
        """Liest TXT-Datei."""
        with open(self.file_path, "r", encoding="utf-8") as f:
            return f.read()
    
    def _extract_text_from_element(self, element) -> str:
        """Extrahiert rekursiv Text aus XML-Element."""
        texts = []
        if element.text:
            texts.append(element.text.strip())
        for child in element:
            texts.append(self._extract_text_from_element(child))
            if child.tail:
                texts.append(child.tail.strip())
        return " ".join(filter(None, texts))
'''

with open('/content/triple-colab/src/file_client.py', 'w') as f:
    f.write(file_client_code)

# openwebui_client.py erstellen
openwebui_client_code = '''"""OpenWebUI-kompatibler Client mit Gemini/OpenAI-Support."""
import requests
import time
import logging
from typing import Optional

logger = logging.getLogger(__name__)

class OpenWebUIClient:
    """Client für OpenAI-kompatible APIs (inkl. Gemini)."""
    
    def __init__(self, api_key: str, base_url: str, model: str, temperature: float = 0.1):
        self.api_key = api_key
        self.base_url = base_url.rstrip("/")
        self.model = model
        self.temperature = temperature
        self.max_retries = 3
        self.retry_delay = 2
    
    def generate_response(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Generiert Antwort mit automatischem Retry."""
        for attempt in range(self.max_retries):
            try:
                if "generativelanguage.googleapis.com" in self.base_url:
                    return self._call_gemini_api(prompt, system_prompt)
                else:
                    return self._call_openai_api(prompt, system_prompt)
            except Exception as e:
                logger.warning(f"Versuch {attempt + 1}/{self.max_retries} fehlgeschlagen: {e}")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay * (attempt + 1))
                else:
                    logger.error(f"Alle Versuche fehlgeschlagen: {e}")
                    return None
    
    def _call_gemini_api(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Ruft Gemini API auf."""
        url = f"{self.base_url}:generateContent?key={self.api_key}"
        payload = {
            "contents": [{"parts": [{"text": prompt}]}],
            "generationConfig": {"temperature": self.temperature}
        }
        if system_prompt:
            payload["systemInstruction"] = {"parts": [{"text": system_prompt}]}
        response = requests.post(url, json=payload, timeout=120)
        response.raise_for_status()
        data = response.json()
        return data["candidates"][0]["content"]["parts"][0]["text"]
    
    def _call_openai_api(self, prompt: str, system_prompt: Optional[str] = None) -> Optional[str]:
        """Ruft OpenAI-kompatible API auf."""
        url = f"{self.base_url}/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": prompt})
        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature
        }
        response = requests.post(url, headers=headers, json=payload, timeout=120)
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"]
'''

with open('/content/triple-colab/src/openwebui_client.py', 'w') as f:
    f.write(openwebui_client_code)

print("✓ Setup abgeschlossen!")
print("✓ Pakete installiert")
print("✓ Code-Module erstellt")

In [ ]:
#@title **API-Konfiguration** { display-mode: "form" }

#@markdown ### Wähle deinen API-Provider:
api_provider = "Gemini (Google)" #@param ["Gemini (Google)", "ChatAI (AcademicCloud)", "Eigene OpenAI-API"]

#@markdown ### API-Schlüssel:
#@markdown **Wichtig:** Gib niemals deinen API-Schlüssel in öffentlichen Notebooks weiter! Lösche ihn vor dem Teilen.
api_key = "" #@param {type:"string"}

#@markdown ### Eigenes Modell (optional):
#@markdown Leer lassen für Standard-Modell des Providers
custom_model = "" #@param {type:"string"}

#@markdown ### Detailgrad der Extraktion:
#@markdown Wie ausführlich sollen die Triples sein? (1=nur Kernaussagen, 10=sehr detailliert)
granularity = 5 #@param {type:"slider", min:1, max:10, step:1}

#@markdown ### Temperatur (Kreativität):
#@markdown Wie kreativ soll die KI antworten? (0=deterministisch, 1=sehr kreativ)
temperature = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}

#@markdown ### Erweiterte Optionen:
#@markdown Datei-Metadaten in Ergebnissen speichern (Dateiname, Datum, etc.):
save_file_metadata = True #@param {type:"boolean"}
#@markdown Debug-Modus: Zeige KI-Antworten bei Fehlern (hilft bei Problemanalyse):
show_debug_output = False #@param {type:"boolean"}

# API-Schlüssel validieren
if not api_key or len(api_key.strip()) < 10:
    raise ValueError(
        "❌ Ungültiger API-Schlüssel!\n\n"
        "Bitte gib einen gültigen API-Schlüssel ein.\n\n"
        "API-Schlüssel erhalten:\n"
        "• Gemini: https://aistudio.google.com/apikey\n"
        "• ChatAI: https://chat-ai.academiccloud.de/\n"
        "• OpenAI: https://platform.openai.com/api-keys"
    )

# Konfiguration basierend auf Provider
if api_provider == "Gemini (Google)":
    if custom_model:
        selected_model = custom_model
        base_url = f"https://generativelanguage.googleapis.com/v1beta/models/{custom_model}"
    else:
        selected_model = "gemini-2.0-flash"
        base_url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash"
elif api_provider == "ChatAI (AcademicCloud)":
    selected_model = custom_model if custom_model else "llama-3.3-70b-instruct"
    base_url = "https://chat-ai.academiccloud.de/v1"
else:
    selected_model = custom_model if custom_model else "gpt-4o-mini"
    base_url = "https://api.openai.com/v1"

# Konfiguration speichern
selected_config = {
    'provider': api_provider,
    'api_key': api_key,
    'base_url': base_url,
    'model': selected_model,
    'temperature': temperature,
    'granularity': granularity,
    'include_metadata': save_file_metadata,
    'verbose': show_debug_output
}

print(f"✓ Konfiguration gespeichert")
print(f"  Provider: {api_provider}")
print(f"  Modell: {selected_model}")
print(f"  Detailgrad: {granularity}")
print(f"  Temperatur: {temperature}")

In [ ]:
#@title **System-Prompt bearbeiten** (optional) { display-mode: "form" }
#@markdown Hier kannst du den System-Prompt anpassen. Nur relevant wenn "Eigenen Prompt verwenden" aktiviert ist.

#@markdown ### Eigenen System-Prompt verwenden:
use_custom_prompt = False #@param {type:"boolean"}

#@markdown ### System-Prompt:
CUSTOM_PROMPT = """Du bist ein Experte für die Analyse historischer Briefe und die Extraktion von Wissensgraphen.

Deine Aufgabe ist es, aus dem gegebenen Brieftext semantische Triples im Format (Subjekt, Prädikat, Objekt) zu extrahieren.

Beachte dabei:
- Extrahiere nur faktische Beziehungen, keine Interpretationen
- Verwende klare, präzise Prädikate
- Normalisiere Entitäten (z.B. "J. W. v. Goethe" -> "Johann Wolfgang von Goethe")
- Berücksichtige historischen Kontext
- Extrahiere sowohl explizite als auch implizite Beziehungen

Antworte ausschließlich im JSON-Format mit folgendem Schema:
{
  "triples": [
    {"subject": "...", "predicate": "...", "object": "..."}
  ]
}""" #@param {type:"string"}

if use_custom_prompt:
    print("✓ Eigener System-Prompt aktiviert")
    print(f"  Länge: {len(CUSTOM_PROMPT)} Zeichen")
else:
    print("ℹ Standard-Prompt wird verwendet")

In [ ]:
#@title **Dateien hochladen** { display-mode: "form" }
#@markdown Klicke auf "Dateien auswählen" und wähle deine XML-Dateien oder ein ZIP-Archiv.

from google.colab import files
import zipfile
import shutil

# Upload-Verzeichnis vorbereiten
upload_dir = '/content/triple-colab/uploads'
if os.path.exists(upload_dir):
    shutil.rmtree(upload_dir)
os.makedirs(upload_dir)

print("Bitte wähle deine Dateien aus...")
uploaded = files.upload()

# Hochgeladene Dateien verarbeiten
uploaded_files = []
for filename, content in uploaded.items():
    filepath = os.path.join(upload_dir, filename)
    with open(filepath, 'wb') as f:
        f.write(content)
    
    # ZIP-Archive entpacken
    if filename.endswith('.zip'):
        print(f"Entpacke {filename}...")
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(upload_dir)
        os.remove(filepath)

# XML-Dateien sammeln
for root, dirs, files_in_dir in os.walk(upload_dir):
    for file in files_in_dir:
        if file.endswith('.xml'):
            uploaded_files.append(os.path.join(root, file))

print(f"\n✓ {len(uploaded_files)} XML-Datei(en) gefunden:")
for f in uploaded_files[:5]:
    print(f"  • {os.path.basename(f)}")
if len(uploaded_files) > 5:
    print(f"  ... und {len(uploaded_files) - 5} weitere")

In [ ]:
#@title **Verarbeitung starten** { display-mode: "form" }
#@markdown Startet die Triple-Extraktion für alle hochgeladenen Dateien.

import json
from datetime import datetime
from file_client import FileClient
from openwebui_client import OpenWebUIClient

# Validierung
if 'selected_config' not in globals():
    raise ValueError(
        "❌ Keine API-Konfiguration gefunden!\n\n"
        "Bitte führe zuerst die Zelle 'API-Konfiguration' aus."
    )

if 'api_key' not in globals() or not api_key:
    raise ValueError(
        "❌ Kein API-Schlüssel gefunden!\n\n"
        "Bitte führe die Zelle 'API-Konfiguration' aus und gib deinen API-Schlüssel ein."
    )

if 'uploaded_files' not in globals() or not uploaded_files:
    raise ValueError(
        "❌ Keine Dateien hochgeladen!\n\n"
        "Bitte führe zuerst die Zelle 'Dateien hochladen' aus."
    )

# Client initialisieren
client = OpenWebUIClient(
    api_key=selected_config['api_key'],
    base_url=selected_config['base_url'],
    model=selected_config['model'],
    temperature=selected_config['temperature']
)

# System-Prompt
system_prompt = CUSTOM_PROMPT if ('use_custom_prompt' in globals() and use_custom_prompt) else None

# Prompt-Template
def create_prompt(text, granularity):
    return f"""Extrahiere semantische Triples aus folgendem historischen Brieftext.

Granularität: {granularity}/10 (1=grob, 10=sehr detailliert)

Text:
{text}

Antworte NUR mit gültigem JSON im Format:
{{
  "triples": [
    {{"subject": "...", "predicate": "...", "object": "..."}}
  ]
}}"""

# Verarbeitung
results = []
output_dir = '/content/triple-colab/output_json'

print(f"Verarbeite {len(uploaded_files)} Datei(en)...\n")

for i, filepath in enumerate(uploaded_files, 1):
    filename = os.path.basename(filepath)
    print(f"[{i}/{len(uploaded_files)}] {filename}")
    
    try:
        # Text einlesen
        file_client = FileClient(filepath)
        text = file_client.read_content()
        
        if not text:
            print(f"  ⚠ Konnte Text nicht extrahieren")
            continue
        
        # API-Anfrage
        prompt = create_prompt(text, selected_config['granularity'])
        response = client.generate_response(prompt, system_prompt)
        
        if not response:
            print(f"  ❌ Keine Antwort erhalten")
            continue
        
        # JSON parsen
        try:
            clean_response = response.strip()
            if clean_response.startswith('```'):
                clean_response = '\n'.join(clean_response.split('\n')[1:-1])
            if clean_response.startswith('json'):
                clean_response = '\n'.join(clean_response.split('\n')[1:])
            
            data = json.loads(clean_response)
            triple_count = len(data.get('triples', []))
            
            # Ergebnis speichern
            output_filename = os.path.splitext(filename)[0] + '.json'
            output_path = os.path.join(output_dir, output_filename)
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            
            results.append({
                'filename': filename,
                'triples': data.get('triples', []),
                'output_path': output_path
            })
            
            print(f"  ✓ {triple_count} Triples extrahiert")
            
        except json.JSONDecodeError as e:
            print(f"  ❌ JSON-Parse-Fehler: {e}")
            if selected_config.get('verbose', False):
                print(f"  Response: {response[:200]}...")
            continue
            
    except Exception as e:
        print(f"  ❌ Fehler: {e}")
        continue

print(f"\n✓ Verarbeitung abgeschlossen!")
print(f"  {len(results)} von {len(uploaded_files)} Dateien erfolgreich verarbeitet")
total_triples = sum(len(r['triples']) for r in results)
print(f"  {total_triples} Triples gesamt extrahiert")

In [ ]:
#@title **Ergebnis visualisieren** { display-mode: "form" }
#@markdown Zeigt den Wissensgraphen für das zuletzt verarbeitete Ergebnis.

import plotly.graph_objects as go
import networkx as nx

if 'results' not in globals() or not results:
    print("❌ Keine Ergebnisse vorhanden. Bitte führe zuerst die Verarbeitung aus.")
else:
    last_result = results[-1]
    triples = last_result['triples']
    
    if not triples:
        print("⚠ Keine Triples zum Visualisieren gefunden.")
    else:
        G = nx.DiGraph()
        
        for triple in triples:
            subj = triple.get('subject', '')
            pred = triple.get('predicate', '')
            obj = triple.get('object', '')
            G.add_edge(subj, obj, label=pred)
        
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        edge_trace = []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_trace.append(
                go.Scatter(
                    x=[x0, x1, None],
                    y=[y0, y1, None],
                    mode='lines',
                    line=dict(width=1, color='#888'),
                    hoverinfo='none'
                )
            )
        
        node_x = []
        node_y = []
        node_text = []
        
        for node in G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            node_text.append(node)
        
        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode='markers+text',
            text=node_text,
            textposition='top center',
            hoverinfo='text',
            marker=dict(
                size=20,
                color='lightblue',
                line=dict(width=2, color='darkblue')
            )
        )
        
        fig = go.Figure(
            data=edge_trace + [node_trace],
            layout=go.Layout(
                title=f"Wissensgraph: {last_result['filename']}",
                showlegend=False,
                hovermode='closest',
                margin=dict(b=0, l=0, r=0, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                height=600
            )
        )
        
        fig.show()
        print(f"\nGraph mit {len(G.nodes())} Entitäten und {len(G.edges())} Beziehungen")

In [ ]:
#@title **Ergebnisse als ZIP herunterladen** { display-mode: "form" }
#@markdown Lädt alle JSON-Ergebnisse als ZIP-Archiv herunter.

from google.colab import files
import zipfile
from datetime import datetime

if 'results' not in globals() or not results:
    print("❌ Keine Ergebnisse vorhanden. Bitte führe zuerst die Verarbeitung aus.")
else:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = f'triple_extraction_results_{timestamp}.zip'
    zip_path = f'/content/{zip_filename}'
    
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for result in results:
            output_path = result['output_path']
            arcname = os.path.basename(output_path)
            zipf.write(output_path, arcname)
    
    print(f"Lade {zip_filename} herunter...")
    files.download(zip_path)
    print(f"✓ Download gestartet ({len(results)} Dateien)")

## Hilfe & Troubleshooting

| Problem | Lösung |
|---------|--------|
| **NameError: api_key not defined** | Führe die Zelle "API-Konfiguration" aus |
| **Ungültiger API-Schlüssel** | Prüfe ob der Schlüssel korrekt kopiert wurde |
| **Rate Limit Error** | Warte kurz und versuche es erneut |
| **Timeout** | Versuche es mit weniger oder kleineren Dateien |
| **Keine Triples extrahiert** | Erhöhe den Detailgrad oder passe den Prompt an |
| **JSON Parse Error** | Aktiviere "Debug-Modus" in der Konfiguration |

**API-Schlüssel erhalten:**
- Gemini: https://aistudio.google.com/apikey
- ChatAI: https://chat-ai.academiccloud.de/
- OpenAI: https://platform.openai.com/api-keys